In [70]:
def parse(driver,l,genes,target_frequency):
    results = []
    count = [0]*len(l)

#claims = driver.find_elements_by_class_name("claims")

#dog = ""

#for x in claims:
#    dog = dog + x.text + "\n"

#print(dog) 
#claim = dog

    #TITLE
    #titleStr = driver.find_element_by_id("title").text
    titleStr = driver.title.split(" - ")[1].strip()
    
    #print(titleStr)
    ## SWITCH CASE
    for x in l:
        for y in x:
            #y=str(y).lower()
            if len(str(y))>=3:
                #print("entered")
                if " "+str(y)+" " in titleStr:
                    count[l.index(x)]+=titleStr.count(" "+str(y)+" ")
                    results.append(x[0])
                    break
                if  " " + str(y) + "." in titleStr:
                    count[l.index(x)]+=titleStr.count(" " + str(y) + ".")
                    results.append(x[0])
                    break
                if  " " + str(y) + "," in titleStr:
                    count[l.index(x)]+=titleStr.count(" " + str(y) + ",")
                    results.append(x[0])
                    break
                if  " " + str(y) + "'" in titleStr:
                    count[l.index(x)]+=titleStr.count(" " + str(y) + "'")
                    results.append(x[0])
                    break
                if  " " + str(y) + "/" in titleStr:
                    count[l.index(x)]+=titleStr.count(" " + str(y) + "/")
                    results.append(x[0])
                    break
                if  "/" + str(y) + " " in titleStr:
                    count[l.index(x)]+=titleStr.count("/" + str(y) + " ")
                    results.append(x[0])
                    break
                if  " " + str(y) + ":" in titleStr:
                    count[l.index(x)]+=titleStr.count(" " + str(y) + ":")
                    results.append(x[0])
                    break
        
        
    #ABSTRACT
    #abstractStr = driver.find_element_by_id("abstract").text
    abstractStr = driver.find_element_by_id("abstract").text

    if not abstractStr:
        abstractStr = driver.find_element_by_tag_name("patent-text").text
    #print(abstractStr)

    for x in l:
        #y=str(y).lower()
        for y in x:
            y=str(y).lower()
            if len(str(y))>=3:
                #print("entered")
                if " "+str(y)+" " in abstractStr:
                    count[l.index(x)]+=abstractStr.count(" "+str(y)+" ")
                elif  " " + str(y) + "." in abstractStr:
                    count[l.index(x)]+=abstractStr.count(" " + str(y) + ".")
                elif  " " + str(y) + "," in abstractStr:
                    count[l.index(x)]+=abstractStr.count(" " + str(y) + ",")
                elif  " " + str(y) + "'" in abstractStr:
                    count[l.index(x)]+=abstractStr.count(" " + str(y) + "'")
                elif  " " + str(y) + "/" in abstractStr:
                    count[l.index(x)]+=abstractStr.count(" " + str(y) + "/")
                elif  "/" + str(y) + " " in abstractStr:
                    count[l.index(x)]+=abstractStr.count("/" + str(y) + " ")
                elif  " " + str(y) + ":" in abstractStr:
                    count[l.index(x)]+=abstractStr.count(" " + str(y) + ":")

        
        
    #CLAIMS
    claims = driver.find_elements_by_class_name("claims")

    claimList=[]
    temp = 0
    for x in claims:
        if temp==10:
            break
        claimList.append(x.text+"\n")
        temp+=1
    claimStr = ""
    for x in claimList:
        claimStr+=x

    #print(claimStr)

    for x in l:
        for y in x:
            #y=str(y).lower()    
            if len(str(y))>=3:
                #print("entered")
                if " "+str(y)+" " in claimStr:
                    count[l.index(x)]+=claimStr.count(" "+str(y)+" ")
                elif  " " + str(y) + "." in claimStr:
                    count[l.index(x)]+=claimStr.count(" " + str(y) + ".")
                elif  " " + str(y) + "," in claimStr:
                    count[l.index(x)]+=claimStr.count(" " + str(y) + ",")
                elif  " " + str(y) + "'" in claimStr:
                    count[l.index(x)]+=claimStr.count(" " + str(y) + "'")
                elif  " " + str(y) + "/" in claimStr:
                    count[l.index(x)]+=claimStr.count(" " + str(y) + "/")
                elif  "/" + str(y) + " " in claimStr:
                    count[l.index(x)]+=claimStr.count("/" + str(y) + " ")
                elif  " " + str(y) + ":" in claimStr:
                    count[l.index(x)]+=claimStr.count(" " + str(y) + ":")
                
    #ADD TO RESULTS
    #temp = count
    #c = 0
    #while c<3:
    #    t = l[l.index(max(count))][len(l[l.index(max(count))])-1]
    #    if t not in results:
    #        results.append(t)
    #        count.remove(max(count))
    #    c+=1
        
    #for y in l:
    #    if count[l.index(y)]>=3 and l[l.index(y)][len(l[l.index(y)])-1] not in results:
    #        results.append(y[len(y)-1])
    
    
    r = sorted(zip(count, l), reverse=True)[:3]
    for x in r:
        if x[0]>=target_frequency:
            results.append(x[1][0])
    print(r)
    #print(r)
    
    #c = 0
    #if len(results)==0:
    #    for y in l:
    #        if count[l.index(y)]!=0 and c<2:
    #            results.append(y[len(y)-1])
    #            count+=1
    
    #for x in count:
    #    print("Target: "+l[count.index(x)][len(l[count.index(x)])-1])
    #    print("#s of times seen: "+str(x)+"\n")
    #list = list[:3]
    return results

In [71]:
from selenium.webdriver.firefox.firefox_binary import FirefoxBinary
from selenium import webdriver
import time
import xlrd
driver = webdriver.Firefox(executable_path=r'C:\Users\jjohn\Desktop\Scraping\geckodriver-v0.29.1-win64\geckodriver.exe') #make sure this exists somewhere in a local, varies from user to user, and copy the path here

driver.get("https://patents.google.com/patent/US20080234138A1/en?q=tp53")

time.sleep(5)

#GENE ARRAY
l = newaliasesfromfile()
gene = genes()
#print(l)

list = parse(driver,l,gene,1)

print("Genes found")
for x in list:
    print(x)

driver.close()

[(6, ['ZNF600', 'Zinc Finger Protein 600', 'KR-ZNF1', 'Zinc Finger Protein KR-ZNF1', 'DKFZp686F06123', 'ZNF600']), (6, ['ZMAT3', 'Zinc Finger Matrin-Type 3', 'PAG608', 'WIG1', 'Zinc Finger Matrin-Type Protein 3', 'P53-Activated Gene 608 Protein', 'WIG-1', 'P53 Target Zinc Finger Protein', 'Zinc Finger Protein WIG-1', 'Zinc Finger Protein WIG1', 'WIG-1/PAG608 Protein', 'MGC10613', 'FLJ12296', 'ZMAT3']), (6, ['WNT16', 'Wnt Family Member 16', 'Wingless-Type MMTV Integration Site Family, Member 16', 'Protein Wnt-16', 'Wingless-Type MMTV Integration Site Family Member 16b', 'WNT16'])]
Genes found
ZNF600
ZMAT3
WNT16


In [46]:
import xlrd
def aliasesfromfile():
    wb = xlrd.open_workbook  # read all genes from excel sheet
    ws = wb.sheet_by_index(0)
    x = len(ws.col_values(0))

    genelist = ws.col_values(0)
    genelist.pop(0)  # genelist = list of all target names
    aliases = []
    for dog in range(x):
        x = ws.row_values(dog)
        x[:] = [y for y in x if y]
        x = x[1:]
        x = str(x)
        aliases.append(x)
    return aliases

In [56]:
import pandas as pd
def newaliasesfromfile():
    pepega = pd.read_excel("Gene_Aliases_All.xlsx")
    #print(pepega["Unnamed: 0"].values[0])
    #print(pepega.values[0])
    poogers = []
    for dog in range(len(pepega.values)):
        peepo = []
        for cow in range(len(pepega.values[dog])-1):
            if pd.isnull(pepega.values[dog][cow]):
                break
            peepo.append(pepega.values[dog][cow])
        poogers.append(peepo)
    return poogers

In [67]:
import pandas as pd
def genes():
    pepega = pd.read_excel("Gene_Aliases_All.xlsx")
    #print(pepega["Unnamed: 0"].values[0])
    #print(pepega.values[0])
    return pepega["Unnamed: 0"]